# Problem Set 3
## By Scott Behmer. January 24, 2018

The following code uses the data file MacroSeries.txt. If that dataset is stored somewhere other than the folder of the jupyter notebook, the import command will have to be changed.

The code in this notebook estimates the parameters of the Brock and Mirman economic model using time series data and maximum likelihood estimation. The cell below loads the relevant function libraries and the dataset.

In [25]:
#Loading Function Libraries
import numpy as np
import scipy.stats as sts
import scipy.optimize as opt

#Importing Data
data = np.loadtxt('MacroSeries.txt', delimiter = ",")

## Part A: First MLE Estimation using wage time series

The code below defines the likelihood function, finds the parameters that maximize that function, and then prints out the results.

The likelihood function takes in the time series data and the four parameters, then uses the following equation to calculate a time series for total factor productivity $z_t$:

$$
z_{t} = ln\left(\frac{w_{t}}{\left(1-\alpha\right)k_{t}^{\alpha}}\right)
$$

where $w_t$ is the wage and $k_t$ is the household savings in time period $t$. From the total factor productivity time series, the error terms in each period are then calculated according to equation 5 of the model:
$$
\epsilon_{t} = z_t - \rho z_{t-1} - \left(1-\rho\right)\mu
$$

Because the model specifies that $\epsilon_t$ is distributed according to a normal with mean zero and variance $\sigma^2$, the likelihood of each epsilon can be calculated using the normal pdf.

Notice that in the optimization function (opt.minimize), an unconstrained algorithm is used even though the parameters of the model have clear bounds. This is because the unconstrained method consistently gave results that were within the parameter bounds, and those results were more robust to changes in initial conditions than they were when using different methods (including constrained optimization). Because the algorithm consistently exits with similar parameter values, over a wide range of initial conditions, I am fairly confident that the resulting estimates represent a global maximum of the likelihood function.

In [27]:
#Creating log likelihood function for part a
def loglik1(data, alpha, rho, mu, sigma):
    z = np.zeros(len(data))
    for i in range(0, len(data)):
        z[i] = np.log(data[i][2]/((1-alpha)*(data[i][1]**alpha)))
    eps = np.zeros(len(data)-1)
    for i in range(0, len(eps)):
        eps[i] = (z[i+1] - rho*z[i] - (1-rho)*mu)
    log_lik = sum(np.log(sts.norm.pdf(eps, loc = 0, scale = sigma)))
    return log_lik

#Defining Criteria Function which will be minimized
def crit(params, args):
    alpha, rho, mu, sigma = params
    xvals = args
    log_lik_val = loglik1(xvals, alpha, rho, mu, sigma)
    neg_log_lik_val = -log_lik_val
    return neg_log_lik_val

#Calculating initial guesses (these were randomly chosen)
alpha0 = 0.5
rho0 = -.5
mu0 = 5
sigma0 = 5

#Finding MLE using the minimize function
params_init = np.array([alpha0, rho0, mu0, sigma0])
results = opt.minimize(crit, params_init, args=(data), method = 'Nelder-Mead');
alpha_MLE, rho_MLE, mu_MLE, sigma_MLE = results.x

#Printing parameter values, log likelihood, and results
print('alpha_MLE=', alpha_MLE, ' rho_MLE=', rho_MLE, ' mu_MLE=', mu_MLE, ' sigma_MLE', sigma_MLE)
print('Maximized log likelihood: ', loglik1(data, alpha_MLE, rho_MLE, mu_MLE, sigma_MLE))
results

C:\Users\Scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


alpha_MLE= 0.462613426588  rho_MLE= 0.716854471488  mu_MLE= 9.45278094824  sigma_MLE 0.0924535646152
Maximized log likelihood:  95.2489398486


 final_simplex: (array([[ 0.46261343,  0.71685447,  9.45278095,  0.09245356],
       [ 0.46260673,  0.71685818,  9.45287494,  0.09245349],
       [ 0.46261643,  0.71685256,  9.45273964,  0.09245346],
       [ 0.46261712,  0.7168482 ,  9.45273066,  0.09245357],
       [ 0.46261593,  0.71685134,  9.45274734,  0.09245381]]), array([-95.24893985, -95.24893985, -95.24893985, -95.24893985, -95.24893985]))
           fun: -95.248939848619997
       message: 'Optimization terminated successfully.'
          nfev: 664
           nit: 393
        status: 0
       success: True
             x: array([ 0.46261343,  0.71685447,  9.45278095,  0.09245356])

The code below runs a constrained optimization using initial conditions that are very close to the parameter estimates that resulted from the unconstrained optimization. The minimization function exits with similar parameter values (although it is not nearly as robust to changes in initial conditions), and it also outputs the inverse hessian matrix, which is used to find the variance covariance matrix. Aside from the second row, the matrix has fairly small elements, which would make me think that the parameters are estimated fairly precisely. However, due to the strange behavior of the constrained optimization algorithm, I am not very confident in the variance-covariance estimates

In [28]:
#To Calculate the variance covariance matrix, I use constrained optimization (it gives the same results, but also outputs the Hessian)

#Defining Parameter Bounds
bnds = ((0, 1), (-1, 1), (0, None), (0, None))

#Our initial guesses are based on the previous result
params_init = np.array([alpha_MLE, rho_MLE, mu_MLE, sigma_MLE])
#results = opt.minimize(crit, params_init, args=(data), bounds = bnds)
results = opt.minimize(crit, .99*params_init, args=(data), bounds = bnds)
alpha_test, rho_test, mu_test, sigma_test = results.x
offdiagneg = np.array([[1, -1, -1, -1], [-1, 1, -1, -1], [-1, -1, 1, -1], [-1, -1, -1, 1]])
H1 = results.hess_inv*offdiagneg
print('alpha_MLE=', alpha_test, ' rho_MLE=', rho_test, ' mu_MLE=', mu_test, ' sigma_MLE', sigma_test)
print(' ')
print('Variance-Covariance Matrix: ')
print(H1)

results

alpha_MLE= 0.469281959298  rho_MLE= 0.711891579044  mu_MLE= 9.35974915569  sigma_MLE 0.0924539630942
 
Variance-Covariance Matrix: 
[[  0.69973953  -0.42064188   0.32922759   0.70990604]
 [-37.60532279  37.25629584 -31.01504232 -41.68706127]
 [ -4.4113059    1.4291051   -0.89926469  -4.2813702 ]
 [  2.50569635  -2.70659005   2.25495359   2.855752  ]]


      fun: -95.248564084581218
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.01020339, -0.00013358, -0.00870699,  0.00180194])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 105
      nit: 16
   status: 0
  success: True
        x: array([ 0.46928196,  0.71189158,  9.35974916,  0.09245396])

## Part B: MLE using interest rate data

The cell below is nearly identical to the one from part A, except the likelihood function calculates the total factor productivity time series $z_t$ using the following equation:

$$
z_{t} = ln\left(\frac{r_{t}}{\alpha k_{t}^{\alpha-1}}\right)
$$

The normally distributed error terms and their likelihoods are then calculated using the same equations as in part A.

Once again, the unconstrained Nelder-Mead optimization outputted more robust results that other methods. However, in this case, there was some variation depending on initial conditions. The code below uses the MLE estimates from part A as initial conditions. In this case, the optimization terminates at parameter values very close to those initial conditions. However, if different initial guesses are used, the optimization terminates at values with mu equal to about 4.8. This is done in the second cell below. While in either case the part B likelihood function is nearly identical (they are within .0000001% of each other), the parameter values with mu equal to 4.8 make the likelihood function from part A very small. Thus, for the rest of the writeup, I use the parameter values with $\mu$ equal to 9.27 as the part B maximum likelihood estimates.

In [29]:
#Creating log likelihood function for part a
def loglik2(data, alpha, rho, mu, sigma):
    z = np.zeros(len(data))
    for i in range(0, len(data)):
        z[i] = np.log(data[i][3]/(alpha*(data[i][1]**(alpha-1))))
    eps = np.zeros(len(data)-1)
    for i in range(0, len(eps)):
        eps[i] = (z[i+1] - rho*z[i] - (1-rho)*mu)
    log_lik = sum(np.log(sts.norm.pdf(eps, 0, sigma)))
    return log_lik

#Defining Criteria Function which will be minimized
def crit2(params, args):
    alpha, rho, mu, sigma = params
    xvals = args
    log_lik_val = loglik2(xvals, alpha, rho, mu, sigma)
    neg_log_lik_val = -log_lik_val
    return neg_log_lik_val

#Calculating initial guesses based on the expected value and variance of the gamma distribution
alpha0 = alpha_MLE
rho0 = rho_MLE
mu0 = mu_MLE
sigma0 = sigma_MLE


#Finding MLE using the minimize function
params_init = np.array([alpha0, rho0, mu0, sigma0])
results = opt.minimize(crit2, params_init, args=(data), method = 'Nelder-Mead')
alpha2_MLE, rho2_MLE, mu2_MLE, sigma2_MLE = results.x

#Printing parameter values, log likelihood, and results
print('alpha_MLE=', alpha2_MLE, ' rho_MLE=', rho2_MLE, ' mu_MLE=', mu2_MLE, ' sigma_MLE', sigma2_MLE)
print(loglik2(data, alpha2_MLE, rho2_MLE, mu2_MLE, sigma2_MLE))
results

alpha_MLE= 0.462613697184  rho_MLE= 0.716853858642  mu_MLE= 9.27982874535  sigma_MLE 0.0924536529651
95.2489398486


 final_simplex: (array([[ 0.4626137 ,  0.71685386,  9.27982875,  0.09245365],
       [ 0.46261309,  0.71685143,  9.27984023,  0.09245358],
       [ 0.46261761,  0.71685131,  9.27975973,  0.0924535 ],
       [ 0.46261034,  0.71685495,  9.27988808,  0.09245342],
       [ 0.46261003,  0.71685757,  9.27989633,  0.0924535 ]]), array([-95.24893985, -95.24893985, -95.24893985, -95.24893985, -95.24893985]))
           fun: -95.248939848586062
       message: 'Optimization terminated successfully.'
          nfev: 255
           nit: 152
        status: 0
       success: True
             x: array([ 0.4626137 ,  0.71685386,  9.27982875,  0.09245365])

As was mentioned above, the cell below shows that when different initial conditions are used, the resulting parameter values are significantly different, while the part B log-likelihood function is nearly unchanged. However, we see that when these alternate parameter values are plugged into the part A log likelihood function, the result is -2524, which is very small compared to its maximum value. Thus, I do not use these alternate parameter values as the part B MLE estimates.

In [31]:
#Setting Initial Guesses
alpha0 = .5
rho0 = -.5
mu0 = 10
sigma0 = 5

#Finding MLE using the minimize function
params_init = np.array([alpha0, rho0, mu0, sigma0])
results = opt.minimize(crit2, params_init, args=(data), method = 'Nelder-Mead')
alpha3_MLE, rho3_MLE, mu3_MLE, sigma3_MLE = results.x

#Printing results
print('alpha_MLE=', alpha3_MLE, ' rho_MLE=', rho3_MLE, ' mu_MLE=', mu3_MLE, ' sigma_MLE', sigma3_MLE)
print('Log likelihood function from part B ', loglik2(data, alpha3_MLE, rho3_MLE, mu3_MLE, sigma3_MLE))
print('Log likelihood function from part A ', loglik1(data, alpha3_MLE, rho3_MLE, mu3_MLE, sigma3_MLE))

C:\Users\Scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':


alpha_MLE= 0.716853194086  rho_MLE= 0.462614082507  mu_MLE= 4.82098092367  sigma_MLE 0.0924536093961
Log likelihood function from part B  95.2489398487
Log likelihood function from part A  -2524.83982813


Now, just as in part A, I also run a constrained optimization in order to calculate the variance-covariance matrix of our MLE estimates. We see that the constrained optimization exits with similar parameter values, and that the variance covariance matrix is small once again, which suggests that the parameters are precisely estimated.

In [32]:
#To Calculate the variance covariance matrix, I use constrained optimization (it gives the same results, but also outputs the Hessian)

#Defining Parameter Bounds
bnds = ((0, 1), (-1, 1), (0, None), (0, None))

#Our initial guesses are based on the previous result
params_init = np.array([alpha2_MLE, rho2_MLE, mu2_MLE, sigma2_MLE])
results = opt.minimize(crit2, 1.01*params_init, args=(data),  bounds = bnds)
alpha_test, rho_test, mu_test, sigma_test = results.x

#Calculating Variance-Covariance matrix
offdiagneg = np.array([[1, -1, -1, -1], [-1, 1, -1, -1], [-1, -1, 1, -1], [-1, -1, -1, 1]])
H2 = results.hess_inv*offdiagneg

#Outputting results
print('alpha_MLE=', alpha_test, ' rho_MLE=', rho_test, ' mu_MLE=', mu_test, ' sigma_MLE', sigma_test)
print(' ')
print('Variance-Covariance Matrix: ')
print(H2)

alpha_MLE= 0.457479383428  rho_MLE= 0.72060610028  mu_MLE= 9.37223309613  sigma_MLE 0.0924537703342
 
Variance-Covariance Matrix: 
[[-0.10077986 -1.42977011 -1.05648508  0.58033507]
 [-0.02600583  0.82507042  0.45348859 -0.45172458]
 [-1.10087738 -0.994668    0.94667695 -0.94661615]
 [-0.2583088  -2.69413274 -1.74086772  1.00932154]]


## Part C: Using the MLE

The code below uses the parameter estimates from parts A and B to estimate the probability that the interest rate in a period is greater than one given that the household savings in that period is 7,500,000 and the total factor productivity from the previous period was 10. Using equation 3 from the model, we find the minimum TFP value in the current period $z^*$ that results in an interest rate greater than one:

$$
z^* = ln\left(\frac{1}{\alpha k_{t}^{\alpha-1}}\right)
$$

Then, using equation 5, we find the minimum error term $\epsilon^*$ which will result in $z_t > z^*$:

$$
\epsilon^* = z^* - \rho z_{t-1} - \left(1-\rho\right)\mu
$$

Because we know that the error terms are normally distributed, we can write:

$$
P\{r_t > 1 \} = P\{\epsilon_t > \epsilon^* \} = 1 - \Phi\left(\frac{epsilon^*}{\sigma}\right)
$$

where $\Phi$ is the standard normal CDF. The function below, called "probinterest", follows these equations to calculate the probability using the parameters from parts A and B. We see that both estimates are very close to one, with the part B estimate being slightly smaller. If I had to choose which estimate to trust more (although it probably does not matter considering how similar they are), I would take the one from part B because those parameters were calibrated using the interest rate time series, which makes me think that they would better predict interest rates.

In [33]:
#This function takes in current period savings, previous period tfp, and parameter values. It outputs the probability of the
 # current period interest rate being larger than one.
def probinterest(kt, zlag, alpha, rho, mu, sigma):
    zstar = np.log(1/(alpha*(kt**(alpha-1))))
    eps = (zstar - rho*zlag - (1-rho)*mu)
    return 1-sts.norm.cdf(eps, 0, sigma)

#Calculating the probability using our parameter values from parts a and b:
print('Probability with part A estimates: ', probinterest(7500000, 10, alpha_MLE, rho_MLE, mu_MLE, sigma_MLE))
print('Probability with part B estimates: ', probinterest(7500000, 10, alpha2_MLE, rho2_MLE, mu2_MLE, sigma2_MLE))

Probability with part A estimates:  0.999999999572
Probability with part B estimates:  0.99999998957
